In [1]:
import math
import numpy as np
import torch.distributed as dist
import torch.nn.functional as F
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
import torch.utils.data
import yaml
from torch.cuda import amp
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm

import test  # import test.py to get mAP after each epoch
from models.yolo import Model
from utils.datasets import create_dataloader
from utils.general import (
    torch_distributed_zero_first, labels_to_class_weights, plot_labels, check_anchors, labels_to_image_weights,
    compute_loss, plot_images, fitness, strip_optimizer, plot_results, get_latest_run, check_dataset, check_file,
    check_git_status, check_img_size, increment_dir, print_mutation, plot_evolution, set_logging, init_seeds,
non_max_suppression)
from utils.google_utils import attempt_download
from utils.torch_utils import ModelEMA, select_device, intersect_dicts
from utils.torch_utils import select_device, time_synchronized

class AttrDict(dict):
    def __init__(self, *args, **kwargs):
        super(AttrDict, self).__init__(*args, **kwargs)
        self.__dict__ = self

In [53]:
exp = 'runs/exp0'
opt_ = check_file(f'{exp}/opt.yaml')
weights = f'{exp}/weights/best.pt'
nc, names = (1, ['coolingTower'])
rank = -1
device = torch.device('cpu')
half = device.type != 'cpu' 
with open(opt_) as f:
    opt = yaml.load(f, Loader=yaml.FullLoader)
opt = AttrDict(opt)
    
    
#Other params
imgsz = opt['img_size'][0]
batch_size = 1# opt['batch_size']
conf_thres = 0.2
iou_thres = 0.6

with open(opt['hyp']) as f:
    hyp = yaml.load(f, Loader=yaml.FullLoader)  # load hyps
    if 'box' not in hyp:
        warn('Compatibility: %s missing "box" which was renamed from "giou" in %s' %
             (opt['hyp'], 'https://github.com/ultralytics/yolov5/pull/1120'))
        hyp['box'] = hyp.pop('giou')
pretrained = weights.endswith('.pt')
if pretrained:
    with torch_distributed_zero_first(rank):
        attempt_download(weights)  # download if not found locally
    ckpt = torch.load(weights, map_location=device)  # load checkpoint
    if hyp.get('anchors'):
        ckpt['model'].yaml['anchors'] = round(hyp['anchors'])  # force autoanchor
    model = Model(opt['cfg'] or ckpt['model'].yaml, ch=3, nc=nc).to(device)  # create
    exclude = ['anchor'] if opt['cfg'] or hyp.get('anchors') else []  # exclude keys
    state_dict = ckpt['model'].float().state_dict()  # to FP32
    state_dict = intersect_dicts(state_dict, model.state_dict(), exclude=exclude)  # intersect
    model.load_state_dict(state_dict, strict=False)  # load
else:
    model = Model(opt['cfg'], ch=3, nc=nc).to(device)  # create

iouv = torch.linspace(0.5, 0.95, 10).to(device)  # iou vector for mAP@0.5:0.95
niou = iouv.numel()

In [54]:
model.eval()
with open('data/coolingTowers128.yaml') as f:
    data = yaml.load(f, Loader=yaml.FullLoader)  # model dict
check_dataset(data)  # check

img = torch.zeros((1, 3, imgsz, imgsz), device=device)  # init img
_ = model(img.half() if half else img) if device.type != 'cpu' else None  # run once
path = data['train']  # path to val/test images
dataloader = create_dataloader(path, imgsz, batch_size, model.stride.max(), opt,
                               hyp=None, augment=False, cache=False, pad=0.5, rect=True)[0]


Scanning labels ../datasets/coolingTowers128/labels/train.cache (471 found, 0 missing, 151 empty, 1 duplicate, for 622 images): 622it [00:00, 20550.76it/s]


In [2]:
import argparse
import os
import shutil
import time
from pathlib import Path

import cv2
import torch
import torch.backends.cudnn as cudnn
from numpy import random

from models.experimental import attempt_load
from utils.datasets import LoadStreams, LoadImages
from utils.general import (
    check_img_size, non_max_suppression, apply_classifier, scale_coords,
    xyxy2xywh, plot_one_box, strip_optimizer, set_logging)
from utils.torch_utils import select_device, load_classifier, time_synchronized


In [71]:
device = select_device('cpu')
model = attempt_load('yolov5s.pt', map_location=device)
imgsz = check_img_size(416, s=model.stride.max())
t0 = time.time()
img = torch.zeros((1, 3, imgsz, imgsz), device=device)  # init img
_ = model(img.half() if half else img) if device.type != 'cpu' else None  # run once


Fusing layers... 


In [82]:
im0.shape

(720, 1280, 3)

In [75]:
dataset = LoadImages('inference/images/')
for path, img, im0, vid_cap in dataset:
    img = torch.from_numpy(img).to(device)
    half = False
    img = img.half() if half else img.float()  # uint8 to fp16/32
    img /= 255.0  # 0 - 255 to 0.0 - 1.0
    if img.ndimension() == 3:
        img = img.unsqueeze(0)

    # Inference
    t1 = time_synchronized()
    pred = model(img, augment=False)[0]
    pred = non_max_suppression(pred, agnostic=False, classes=None, iou_thres=0.45,conf_thres=0.4)
    for i, det in enumerate(pred):
        if det is not None and len(det):
                    # Rescale boxes from img_size to im0 size
                    det[:, :4] = scale_coords(img.shape[2:], det[:, :4], im0.shape).round()

image 1/2 /home/martinesguerra/Documents/repos/yolov5/inference/images/bus.jpg: image 2/2 /home/martinesguerra/Documents/repos/yolov5/inference/images/zidane.jpg: 

In [86]:
det

tensor([[7.47000e+02, 4.00000e+01, 1.15000e+03, 7.12000e+02, 8.68075e-01, 0.00000e+00],
        [1.16000e+02, 1.97000e+02, 1.00300e+03, 7.12000e+02, 7.97377e-01, 0.00000e+00],
        [4.24000e+02, 4.30000e+02, 5.17000e+02, 7.20000e+02, 7.78108e-01, 2.70000e+01]])